In [1]:
from glob import glob
import pandas as pd
import numpy as np
from scipy import stats  
import matplotlib.pyplot as plt
import japanize_matplotlib

In [2]:
files = glob("../csv/演習問題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/演習問題_データアナリティクス基礎\第10章演習問題_乱数で発生させて分布の基本統計量.xlsx
1 ../csv/演習問題_データアナリティクス基礎\第2章演習問題_クレームデータ.xlsx
2 ../csv/演習問題_データアナリティクス基礎\第3章演習問題_職場ストレス.xlsx
3 ../csv/演習問題_データアナリティクス基礎\第4章演習問題_児童のネット利用時間.xlsx
4 ../csv/演習問題_データアナリティクス基礎\第5章演習問題_生鮮野菜の購入額.xlsx
5 ../csv/演習問題_データアナリティクス基礎\第7章演習問題_授業効果の検証.xlsx
6 ../csv/演習問題_データアナリティクス基礎\第8章演習問題1_健康食品・化粧品販売額.xlsx
7 ../csv/演習問題_データアナリティクス基礎\第8章演習問題2_百貨店の衣料品売上高.xlsx
8 ../csv/演習問題_データアナリティクス基礎\第9章演習問題_自由記述データ＆形態素解析データ.xlsx


In [5]:
df = pd.read_excel(files[2])
print(df.columns)
df.columns = ['id', 'gender', 'years', 'stress']
df.info()
df.head()

Index(['回答者\n番号', '性別', '勤続年数', '強いストレス'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      440 non-null    int64 
 1   gender  440 non-null    object
 2   years   440 non-null    object
 3   stress  440 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.9+ KB


,id,gender,years,stress
0,1,男,1年,ない
1,2,男,1年,ない
2,3,女,2年,ある
3,4,男,1年,ある
4,5,男,1年,ない


強いストレスの人数 199

In [22]:
df.groupby(['stress']).count()

,id,gender,years
stress,,,
ある,199,199,199
ない,241,241,241


入社一年目で強いストレスの割合: 25.5%

In [31]:
df_1st = df.query('years == "1年"')
df_1st.groupby('stress').count() / df_1st.count()|

,gender,id,stress,years
stress,,,,
ある,0.254808,0.254808,NaN,0.254808
ない,0.745192,0.745192,NaN,0.745192


1年目かつ「ある」の男女の割合の差: 67.6%

In [177]:
gender_stress = df_1st.pivot_table(index='gender', columns='stress', values='id', aggfunc='count')
gender_stress['total'] = gender_stress.apply(lambda x: sum(x), axis=1)
display(gender_stress)

(gender_stress['ある'] / gender_stress['total'])[0] - (gender_stress['ある'] / gender_stress['total'])[1]

stress,ある,ない,total
gender,,,
女,25,5,30
男,28,150,178


0.6760299625468165

勤続年数を質的変数とし勤続年数と強いストレスの関連がないと仮定したときの期待度数: 94.1

In [126]:
observation = df.pivot_table(index='stress', columns='years', values='id', aggfunc='count')
observation

years,1年,2年,3年
stress,,,
ある,53,112,34
ない,155,42,44


In [127]:
observation_total = observation.copy()
observation_total.loc['total'] = observation_total.sum()
observation_total['total'] = observation_total.apply(lambda x : sum(x), axis=1)
observation_total

years,1年,2年,3年,total
stress,,,,
ある,53,112,34,199
ない,155,42,44,241
total,208,154,78,440


In [128]:
total_ratio = observation.sum() / observation.sum().sum()
yes = total_ratio * observation.loc['ある'].sum()
no = total_ratio * observation.loc['ない'].sum()

expectation = pd.DataFrame([yes, no,], index=["ある", "ない"])
expectation

years,1年,2年,3年
ある,94.072727,69.65,35.277273
ない,113.927273,84.35,42.722727


$\chi^2$ 仮説検定: $H_0: 関連がない, H_1: 関連がある$
- 棄却域 chi^2値 > 9.21
- chi^2値 = 79.8
- 以上より有意となり勤続年数とストレスには関連性があると言える

In [142]:
# alphaパーセント点のchi^2値
alpha = 0.01
dof = 2
chi_2 = stats.chi2.ppf(1-alpha, dof)
print(f'{chi_2}')

9.21034037197618


In [136]:
# chi^2 統計量
((observation - expectation)**2 / expectation).sum().sum()

79.83782087370571